In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Read the data
df_train = pd.read_csv("train.csv", index_col='id')
df_test = pd.read_csv("test.csv", index_col='id')
submission = pd.read_csv("sample_submission.csv", index_col='id')
# Separate target from predictors
df_target = df_train.pop('target')

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(df_train, df_target, train_size=0.8, test_size=0.2, random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitary)
categorical_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and X_train[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train[my_cols].copy()
X_valid = X_valid[my_cols].copy()

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
'''
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])
'''

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        # ('cat', categorical_transformer, categorical_cols)
    ])

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

def get_score(n_estimators):
    my_pipeline = Pipeline(steps=[
        ('preprocessor', SimpleImputer()),
        ('model', RandomForestRegressor(n_estimators, random_state=0))
    ])
    scores = -1 * cross_val_score(my_pipeline, df_train, df_target,
                                  cv=3,
                                  scoring='neg_mean_absolute_error')
    rmse_scores = np.sqrt(scores)
    return rmse_scores.mean()

In [ ]:
results = {}
for i in range(1,9):
    results[400+50*i] = get_score(400+50*i)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(list(results.keys()), list(results.values()))
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

n_estimators_best = 400

rf_model = RandomForestRegressor(n_estimators=n_estimators_best, random_state=0)
rf_model.fit(df_train, df_target)
submission['target'] = rf_model.predict(df_test)
submission.to_csv('Tabular_Jan2021_submission_2.csv')